# Batch Prediction for Demand Forecasting

This notebook uses the trained XGBoost model to generate demand predictions for a specified period. The key feature of this process is its **iterative nature**. The model predicts demand for one week at a time, and the output of that prediction is then used as an input feature (a sales lag) for the following week. This simulates a real-world scenario where recent sales data informs future forecasts.

## 1. Loading Libraries and Modules

In [44]:
# Core libraries
import numpy as np
import pandas as pd
import pickle
import yaml
import os
import utils_gpu

# Set pandas display options
pd.options.display.max_columns = None

print("Libraries loaded successfully.")

Libraries loaded successfully.


## 2. Loading Model Artifacts and Configuration

We need to load two key artifacts from the training phase:
1.  **The trained `pipeline.pkl`**: This file contains the complete Scikit-learn pipeline, including all preprocessing steps and the final XGBoost model.
2.  **The `model_params.yml` file**: This configuration file contains the list of features the model was trained on, ensuring we use the exact same features for prediction.

In [45]:
# --- Configuration ---
# IMPORTANT: Update this path to point to the directory where your trained model is saved.
# This path should contain a 'model/pipeline.pkl' and a 'params/exeperiment_params.json' or the original 'model_params.yml'
MODEL_OUTPUT_PATH = '../02_models/outputs/202509191639_XGBOOST/' #<-- PLEASE UPDATE THIS PATH
PARAMS_FILE_PATH = '../02_models/model_params.yml'

# --- Load Model ---
model_path = os.path.join(MODEL_OUTPUT_PATH, 'model', 'pipeline.pkl')
print(f"Loading model from: {model_path}")
with open(model_path, 'rb') as f:
    pipeline = pickle.load(f)
print("Model pipeline loaded successfully.")

# --- Load Parameters ---
print(f"Loading parameters from: {PARAMS_FILE_PATH}")
with open(PARAMS_FILE_PATH, "r") as f:
    configs = yaml.safe_load(f)

FEATURE_COLUMNS = configs['FEATURES']
print(f"Model expects {len(FEATURE_COLUMNS)} features.")

Loading model from: ../02_models/outputs/202509191639_XGBOOST/model/pipeline.pkl
Model pipeline loaded successfully.
Loading parameters from: ../02_models/model_params.yml
Model expects 24 features.


/tmp/ipython-input-3428952202.py:11: UserWarning: [06:57:02] WARNING: /workspace/src/gbm/gbtree.cc:384: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  pipeline = pickle.load(f)
/tmp/ipython-input-3428952202.py:11: UserWarning: [06:57:02] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  pipeline = pickle.load(f)
/tmp/ipython-input-3428952202.py:11: UserWarning: [06:57:02] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  pipeline = pickle.load(f)


## 3. Loading and Preparing Production Data

In [46]:
DATA_PATH = '../../data/processed/processed_production.parquet'

print(f"Loading production data from: {DATA_PATH}")
df_prod = pd.read_parquet(DATA_PATH)

# Ensure data types are correct and sort the data chronologically
df_prod['week_of_year'] = df_prod['week_of_year'].astype(int)
df_prod = df_prod.sort_values(["week_of_year", "internal_product_id", "internal_store_id"]).reset_index(drop=True)

print(f"The production dataset contains {df_prod.shape[0]} rows and {df_prod.shape[1]} columns.")
df_prod.head()

Loading production data from: ../../data/processed/processed_production.parquet
The production dataset contains 1505465 rows and 27 columns.


,internal_product_id,internal_store_id,distributor_id,premise,categoria_pdv,zipcode,tipos,label,subcategoria,marca,fabricante,month,week_of_year,city,holiday,previous_month_quantity_sum,previous_month_gross_value_sum,previous_month_net_value_sum,previous_month_gross_profit_sum,previous_month_discount_sum,quantity_lag1,quantity_lag2,quantity_lag3,quantity_lag4,quantity_lag5,discount_rate_month,profit_margin_month
0,1000423277513436210,4596337970914199140,4,On Premise,Restaurant,30720,Draft,Core,Lager / Pilsner,Reformation A Cold One 97.1 Pilsner,Reformation Brewery,1,0,Dalton,0,1.0,99.000000,93.500000,28.500000,0.00,1.0,NaN,NaN,NaN,NaN,0.000000,0.287879
1,1000423277513436210,5480361925742746763,4,On Premise,Bar,30540,Draft,Core,Lager / Pilsner,Reformation A Cold One 97.1 Pilsner,Reformation Brewery,1,0,Ellijay,0,1.0,99.000000,93.500000,28.500000,0.00,1.0,NaN,NaN,NaN,NaN,0.000000,0.287879
2,1000423277513436210,7360454609632053666,4,On Premise,Restaurant,30143,Draft,Core,Lager / Pilsner,Reformation A Cold One 97.1 Pilsner,Reformation Brewery,1,0,Jasper,0,3.0,297.000000,280.500000,85.500000,0.00,1.0,1.0,1.0,NaN,NaN,0.000000,0.287879
3,1000817480846862155,7347904315150057956,6,Off Premise,Package/Liquor,80521,Package,Core,Other Package,Bud Light Seltzer Peach Tea,AB Anheuser Busch Inc,1,0,Fort Collins,0,1.0,36.799999,36.565624,13.113625,0.00,1.0,NaN,NaN,NaN,NaN,0.000000,0.356349
4,1009179103632945474,1001371918471115422,4,Off Premise,Convenience,30175,Package,Core,Lager,Busch,AB Anheuser Busch Inc,1,0,Talking Rock,0,7.0,145.424997,124.147261,30.459262,12.95,2.0,2.0,2.0,1.0,NaN,0.089049,0.209450


### 3.1 Applying Manual Pre-Pipeline Transformations

Here, we replicate the manual data cleaning steps that were performed during training *before* the main preprocessing pipeline was applied. This ensures the prediction data has the exact same structure and format as the training data.

In [47]:
print("Applying manual pre-processing steps to match training data format...")

# 1. Handle '_sum' columns: Fill NaN values with 0
print("Filling NaN values in '_sum' columns with 0.")
cols_sum = [col for col in df_prod.columns if 'sum' in col]
df_prod[cols_sum] = df_prod[cols_sum].fillna(0)

# 2. Handle 'premise' column
# In training, rows with NaN 'premise' were dropped. We replicate that here.
initial_rows = len(df_prod)
df_prod.dropna(subset=['premise'], inplace=True)
if initial_rows > len(df_prod):
    print(f"Dropped {initial_rows - len(df_prod)} rows with missing 'premise' value.")

# Map 'premise' string values to integer values
print("Mapping 'premise' column to integers.")
df_prod['premise'] = df_prod['premise'].map({'Off Premise': 1, 'On Premise': 0}).astype(int)

print("Manual pre-processing complete.")

Applying manual pre-processing steps to match training data format...
Filling NaN values in '_sum' columns with 0.
Mapping 'premise' column to integers.
Manual pre-processing complete.


## 4. Iterative Prediction Loop

This is the core of the notebook. We loop through each week from 1 to 5. In each iteration, we:

1.  **Predict**: Use the model to predict the quantity for the `current_week`.
2.  **Store**: Save the rounded predictions in a new `quantity` column.
3.  **Update Lags**: If it's not the last week, we update the `quantity_lag` features for the `next_week`. The prediction we just made for `week_n` becomes `quantity_lag1` for `week_n+1`. The old `quantity_lag1` becomes `quantity_lag2`, and so on.

In [48]:
print("Starting iterative prediction process...")

# Initialize the quantity column that will hold our predictions
df_prod['quantity'] = 0
df_prod = df_prod.sort_values(["week_of_year", "internal_product_id", "internal_store_id", "distributor_id"]).reset_index(drop=True)

weeks_to_predict = sorted(df_prod['week_of_year'].unique())
print(f"Will predict for weeks: {weeks_to_predict}")

for i, current_week in enumerate(weeks_to_predict):
    print(f"\n--- Predicting for Week {current_week} ---")

    # --- 1. PREDICT ---
    # Select data for the current week
    week_mask = df_prod['week_of_year'] == current_week
    df_current_week = df_prod[week_mask]

    if df_current_week.empty:
        print(f"No data found for week {current_week}. Skipping.")
        continue

    # Ensure we only use the features the model was trained on
    X_pred = df_current_week[FEATURE_COLUMNS]

    # Make predictions
    predictions = pipeline.predict(X_pred)

    # --- 2. STORE ---
    # Round predictions to nearest whole number, as we can't sell fractions of products
    # and ensure they are non-negative.
    rounded_predictions = np.round(predictions).astype(int)
    rounded_predictions[rounded_predictions < 0] = 0

    # Store the predictions in our main dataframe
    df_prod.loc[week_mask, 'quantity'] = rounded_predictions
    print(f"Stored {len(rounded_predictions)} predictions for week {current_week}.")

    # --- 3. UPDATE LAGS ---
    # Check if this is not the last week in our prediction sequence
    if current_week < weeks_to_predict[-1]:
        next_week = weeks_to_predict[i + 1]
        print(f"Updating lag features for next week ({next_week})...")

        # Create a map of (product, store) -> predicted_quantity for the current week
        prediction_map = df_prod[week_mask].set_index(['internal_product_id', 'internal_store_id', 'distributor_id'])['quantity']

        # Select data for the next week
        next_week_mask = df_prod['week_of_year'] == next_week
        df_next_week_idx = df_prod[next_week_mask].index

        # Shift all existing lag features one week back
        for lag in range(4, 0, -1):
            df_prod.loc[
                df_prod.loc[next_week_mask].index.tolist(),
                f'quantity_lag{lag+1}'
            ] = df_prod.loc[
                df_prod.loc[week_mask].index.tolist(),
                f'quantity_lag{lag}'
            ].values
            # df_prod.loc[week_mask, f'quantity_lag{lag}']

        # The new 'quantity_lag1' for the next week is the prediction from the current week
        # We use a multi-index to map the values correctly
        next_week_multi_index = df_prod.loc[next_week_mask].set_index(['internal_product_id', 'internal_store_id', 'distributor_id']).index
        new_lag1_values = next_week_multi_index.map(prediction_map)

        # Assign the new lag values back to the main dataframe
        df_prod.loc[next_week_mask, 'quantity_lag1'] = new_lag1_values.fillna(0) # Fill with 0 if a product-store didn't exist in the previous week

        print(f"Lag features updated for week {next_week}.")

print("\nIterative prediction process completed.")

Starting iterative prediction process...
Will predict for weeks: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]

--- Predicting for Week 0 ---


/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Stored 301093 predictions for week 0.
Updating lag features for next week (1)...
Lag features updated for week 1.

--- Predicting for Week 1 ---


/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Stored 301093 predictions for week 1.
Updating lag features for next week (2)...
Lag features updated for week 2.

--- Predicting for Week 2 ---


...

Stored 301093 predictions for week 2.
Updating lag features for next week (3)...
Lag features updated for week 3.

--- Predicting for Week 3 ---


/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Stored 301093 predictions for week 3.
Updating lag features for next week (4)...
Lag features updated for week 4.

--- Predicting for Week 4 ---


/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Stored 301093 predictions for week 4.

Iterative prediction process completed.


## 5. Format and Save Final Results

Finally, we format the output DataFrame to match the required schema and save it as a Parquet file.

In [49]:
print("Formatting final results...")

# Define the final column mapping and order
FINAL_COLUMNS = {
    'week_of_year': 'semana',
    'internal_store_id': 'pdv',
    'internal_product_id': 'produto',
    'quantity': 'quantidade'
}

# Create the final dataframe
df_final = df_prod[list(FINAL_COLUMNS.keys())].copy()
df_final.rename(columns=FINAL_COLUMNS, inplace=True)

# Add +1 to weeks to ensure the correct format
df_final['semana'] = df_final['semana'] + 1

# Group them all to ensure non-duplicated
df_final = df_final.groupby(['semana', 'pdv', 'produto'])['quantidade'].sum().reset_index()

# Cast all columns to integer type as requested
for col in df_final.columns:
    df_final[col] = df_final[col].astype(int)

# --- Save Results ---
OUTPUT_FILENAME = '../../data/processed/predictions_january_2023.parquet'
df_final.to_parquet(OUTPUT_FILENAME, index=False)

print(f"Final predictions saved to '{OUTPUT_FILENAME}'")
print(f"Final dataframe shape: {df_final.shape}")
df_final.head()

Formatting final results...
Final predictions saved to '../../data/processed/predictions_january_2023.parquet'
Final dataframe shape: (1500000, 4)


,semana,pdv,produto,quantidade
0,1,1001371918471115422,1009179103632945474,2
1,1,1001371918471115422,1029370090212151375,3
2,1,1001371918471115422,1120490062981954254,11
3,1,1001371918471115422,1394381856358939027,10
4,1,1001371918471115422,1454838625590783593,4
